In [108]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import math

In [109]:
data_cols = ['user id', 'movie id', 'rating', 'timestamp']

In [110]:
item_cols = ['movie id', 'movie title', 'release date',
'video release date', 'IMDb URL', 'unknown', 'Action',
'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime',
'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [111]:
user_cols = ['user id', 'age', 'gender', 'occupation', 'zip code']

In [112]:
os.chdir("C://Users//37718890//Downloads/ml-100k/")

In [113]:
users = pd.read_csv('u.user', sep= '|', names=user_cols, encoding= 'latin-1')

item = pd.read_csv('u.item', sep='|', names=item_cols, encoding='latin-1')

data = pd.read_csv('u.data', sep='\t', names=data_cols, encoding='latin-1')

In [114]:
#Sort the DataFrame by User ID

utrain = (data.sort_values('user id'))[:99832]

print(utrain)

       user id  movie id  rating  timestamp
66567        1        55       5  875072688
62820        1       203       4  878542231
10207        1       183       5  875072262
9971         1       150       5  876892196
22496        1        68       4  875072688
...        ...       ...     ...        ...
73676      942       479       4  891283118
67222      942       604       4  891283139
95675      942       478       5  891283017
85822      942       659       5  891283161
68192      942       487       4  891282985

[99832 rows x 4 columns]


In [115]:
print(utrain.tail())

       user id  movie id  rating  timestamp
73676      942       479       4  891283118
67222      942       604       4  891283139
95675      942       478       5  891283017
85822      942       659       5  891283161
68192      942       487       4  891282985


In [116]:
utest = (data.sort_values('user id'))[99833:]
print(utest)

       user id  movie id  rating  timestamp
91841      943       132       3  888639093
91810      943       204       3  888639117
77956      943        94       4  888639929
87415      943        53       3  888640067
77609      943       124       3  875501995
...        ...       ...     ...        ...
96823      943       427       4  888639147
70902      943        12       5  888639093
84518      943       284       2  875502192
72321      943        62       3  888640003
70713      943       210       4  888639147

[167 rows x 4 columns]


In [117]:
print(utest.head())

       user id  movie id  rating  timestamp
91841      943       132       3  888639093
91810      943       204       3  888639117
77956      943        94       4  888639929
87415      943        53       3  888640067
77609      943       124       3  875501995


### Convert them to a NumPy Array for an ease of iteration!

In [118]:
utrain = utrain.to_numpy()

utrain

array([[        1,        55,         5, 875072688],
       [        1,       203,         4, 878542231],
       [        1,       183,         5, 875072262],
       ...,
       [      942,       478,         5, 891283017],
       [      942,       659,         5, 891283161],
       [      942,       487,         4, 891282985]], dtype=int64)

In [119]:
utest = utest.to_numpy()

utest

array([[      943,       132,         3, 888639093],
       [      943,       204,         3, 888639117],
       [      943,        94,         4, 888639929],
       [      943,        53,         3, 888640067],
       [      943,       124,         3, 875501995],
       [      943,       672,         5, 888640125],
       [      943,      1047,         2, 875502146],
       [      943,       541,         4, 888639954],
       [      943,       226,         4, 888639660],
       [      943,       449,         1, 888693158],
       [      943,       470,         4, 888639814],
       [      943,        11,         4, 888639000],
       [      943,       526,         4, 888639523],
       [      943,       549,         1, 888639772],
       [      943,        28,         4, 875409978],
       [      943,       184,         5, 888639247],
       [      943,        22,         4, 888639042],
       [      943,      1188,         3, 888640250],
       [      943,       196,         5, 88863

Ici on décide de spliter le dataset en deux jeux de données

utrain : jeu de données d'entrainement

utest : jeu de données de test


utrain : contient les ratings des utilisateurs ayant un id allant de 1 à 942

utest : contient un seul id d'utilisateur qui est le 943 pour le test

Et on les convertit en des types Array pour une opération d'itération facile

In [120]:
len(utest)

167

In [121]:
len(utrain)

99832

Next, Create a users_list which is a list of users that contains a list of movies rated by him. 

In [122]:
users_list = []
for i in range(1,943) :
    list_ = []
    for j in range (0, len(utrain)) :
        if(utrain[j][0]==i) : 
            list_.append(utrain[j])
        else : 
            break
    utrain = utrain[j:]
    users_list.append(list_)

In [123]:
len(users_list)

942

Define a Function by the Name of EucledianScore. The purpose of the EucledianScore is to
measure the similarity between two users based on their ratings given to movies that they have
both in common. 

In [124]:
def EuclideanScore (train_user, test_user):
    sum = 0
    count = 0
    for i in test_user :
        score = 0
        for j in train_user : 
            if((int(i[1])) == int(j[1])) :
                score = ((float(i[2]) - float(j[2]))* (float(i[2]) - float(j[2])))
                count = count + 1
                sum = sum + score
    if(count < 4) :
        sum = 1000000
    else :
        sum = math.sqrt(sum)
        sum = sum / count
    return sum

In [125]:
score_list = []

for i in range(0,942) :
    score_list.append([i+1, EuclideanScore(users_list[i], utest)])

In [126]:
score_list

[[1, 0.16132629945515797],
 [2, 0.2939723678960656],
 [3, 1000000],
 [4, 0.4330127018922193],
 [5, 0.18949728622879575],
 [6, 0.19899748742132398],
 [7, 0.15040632653673908],
 [8, 0.2664693550105965],
 [9, 0.6123724356957945],
 [10, 0.19509557590325882],
 [11, 0.20979404812458594],
 [12, 0.3165265169252399],
 [13, 0.15962217272848064],
 [14, 0.24222607082590245],
 [15, 0.4807401700618652],
 [16, 0.21066252211737158],
 [17, 0.5091750772173156],
 [18, 0.15465542839437055],
 [19, 0.7905694150420949],
 [20, 0.3640054944640259],
 [21, 0.3330866937632457],
 [22, 0.2028251065252667],
 [23, 0.19543398999264291],
 [24, 0.20655239475589834],
 [25, 0.24686398735912787],
 [26, 0.2602082499332666],
 [27, 0.5890150893739515],
 [28, 0.23035022137995856],
 [29, 0.4330127018922193],
 [30, 0.31622776601683794],
 [31, 1000000],
 [32, 0.2727272727272727],
 [33, 1000000],
 [34, 1000000],
 [35, 1000000],
 [36, 1000000],
 [37, 0.17407765595569782],
 [38, 0.3429345281202573],
 [39, 1000000],
 [40, 1000000],
 

But what if the users have just one movie in common? Having more movies in common is a
great sign of similarity. So, if users have less than 4 movies in common then we assign them a
high EucledianScore.

Next, we will iterate over users_list and find the similarity of the users to the test_user by
means of this function and append the EucledianScore along with the User ID to a separate
list score_list.

In [127]:
score = pd.DataFrame(score_list, columns=['user id', 'Euclidean Score'])

score = score.sort_values(by = 'Euclidean Score')

print(score)

     user id  Euclidean Score
91        92         0.104137
275      276         0.105318
428      429         0.114031
888      889         0.115134
434      435         0.116369
..       ...              ...
723      724   1000000.000000
204      205   1000000.000000
728      729   1000000.000000
680      681   1000000.000000
712      713   1000000.000000

[942 rows x 2 columns]


We then convert it first to a DataFrame, sort it by the EucledianScore, and finally convert it to
a NumPy Array score_matrix for the ease of iteration.

In [128]:
score_matrix = score.to_numpy()

In [129]:
score_matrix

array([[9.20000000e+01, 1.04137337e-01],
       [2.76000000e+02, 1.05317715e-01],
       [4.29000000e+02, 1.14031317e-01],
       ...,
       [7.29000000e+02, 1.00000000e+06],
       [6.81000000e+02, 1.00000000e+06],
       [7.13000000e+02, 1.00000000e+06]])

We get the user having the lowest Eucledian score and hence the highest similarity. So now
we need to obtain the list of movies that are not common between the two users.

In [130]:
user = int(score_matrix[0][0])

common_list = []

full_list = []

for i in utest:
    for j in users_list[user-1]:
        if(int(i[1]) == int(j[1])) :
            common_list.append(int(j[1]))
        full_list.append(j[1])
common_list = set(common_list)
full_list = set(full_list)

recommendation = full_list.difference(common_list)

In [131]:
recommendation

{1,
 4,
 5,
 7,
 8,
 13,
 15,
 25,
 29,
 32,
 39,
 40,
 43,
 44,
 46,
 47,
 48,
 49,
 63,
 65,
 66,
 71,
 77,
 78,
 81,
 82,
 85,
 87,
 88,
 89,
 91,
 93,
 95,
 101,
 102,
 106,
 108,
 109,
 115,
 116,
 118,
 120,
 123,
 125,
 129,
 134,
 135,
 143,
 144,
 145,
 147,
 148,
 149,
 153,
 154,
 155,
 156,
 157,
 159,
 160,
 164,
 167,
 169,
 171,
 175,
 176,
 179,
 180,
 183,
 189,
 190,
 191,
 198,
 199,
 203,
 208,
 209,
 212,
 214,
 215,
 220,
 222,
 223,
 225,
 235,
 238,
 240,
 241,
 243,
 245,
 246,
 248,
 249,
 250,
 252,
 257,
 258,
 260,
 265,
 268,
 271,
 273,
 276,
 278,
 288,
 289,
 291,
 294,
 295,
 304,
 307,
 313,
 322,
 328,
 363,
 364,
 368,
 369,
 370,
 376,
 382,
 383,
 396,
 408,
 409,
 410,
 411,
 418,
 425,
 428,
 432,
 433,
 436,
 451,
 452,
 453,
 455,
 456,
 463,
 466,
 474,
 476,
 500,
 501,
 504,
 515,
 518,
 521,
 527,
 528,
 531,
 540,
 551,
 552,
 554,
 558,
 561,
 575,
 577,
 582,
 583,
 587,
 591,
 596,
 597,
 619,
 620,
 627,
 628,
 631,
 636,
 640,
 642,


In [132]:
item_list = (((pd.merge(item, data).sort_values(by = 'movie id')).groupby('movie title')))['movie id', 'movie title', 'rating']

#item_list = pd.merge(item, data).sort_values(by='movie id').groupby('movie title')[['movie id', 'movie title', 'rating']]

item_list = item_list.mean()


item_list['movie title'] = item_list.index

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

Then, we need to create a compiled list of the movies along with their mean ratings. Merge the
item and data files.Then groupby movie titles, select the columns you need and then find the
mean ratings of each movie. Then, express the dataframe as a NumPy Array.